In [3]:
import xgboost
import pandas as pd
import os
import utils

In [12]:
FEATURE_COLUMNS = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]',
       'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]', 'FOI_hits_N']

In [6]:
DATA_PATH = "../MuID_Run_III/extractor"

Reading data takes around 5 minutes. We need to find a better format for the variable-length arrays in FOI*. Possibiities:
1. `pandas.to_hdf` failed with something related to too high number of entries
2. `pandas.to_picke` is possible, but also slow and platform-dependent
3. CERN ROOT should do the job, but will be unfamiliar to participants

In [7]:
train = pd.read_csv(os.path.join(DATA_PATH, "train.csv.gz"))

In [8]:
test = pd.read_csv(os.path.join(DATA_PATH, "test_public.csv.gz"))

In [9]:
train.head()

,Index,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,...,FOI_hits_Y,FOI_hits_Z,FOI_hits_DX,FOI_hits_DY,FOI_hits_DZ,FOI_hits_T,FOI_hits_DT,FOI_hits_S,weight,label
0,0,25.0,8.0,7.0,14.0,1.400000,1.25,1.142857,1.571428,8,...,[3166.4326 3430.8433 3430.8433 3678.9775 3928....,[15330.503 16619.656 16619.656 17813.35 19022...,[ 25.5 27.5 27.5 118. 126. ],[126.27855 136.27849 136.27849 146.27841 156.2...,[33.95495 33.99096 33.99096 34.026966 34.06...,[7. 5. 1. 6. 9.],[ 0. 3. -1. 0. 0.],[1. 2. 2. 3. 4.],0.145543,1
1,1,27.0,5.0,5.0,8.0,2.148148,1.00,1.800000,1.250000,8,...,[-318.64474 -344.5297 -369.2936 -392.42264],[15317.953 16522.06 17714.77 18922.787],[12.75 13.75 59. 63. ],[63.078957 68.078926 73.078896 78.07886 ],[33.727364 33.74537 33.763374 33.78138 ],[5. 6. 2. 5.],[0. 0. 0. 0.],[1. 2. 3. 4.],-0.571444,1
2,2,22.0,5.0,2.0,18.0,2.636364,3.20,1.000000,1.500000,8,...,[ 604.1468 664.03204 -414.53174 449.50488 ...,[15405.276 15323.392 16605.809 17516.717 18724...,[76.5 76.5 3.4583333 14.833333 1...,[ 31.479162 31.479162 136.27849 18.179249 ...,[33.613575 33.613575 33.99096 33.56568 33.57...,[14. 1. 7. 7. 8. 1. 9. 3. 2.],[14. 1. 7. 7. 8. 1. -1. 0. 0.],[1. 1. 2. 3. 4. 4. 4. 4. 4.],0.217224,0
3,3,25.0,15.0,5.0,4.0,2.040000,1.80,1.800000,1.000000,8,...,[-797.96094 -721.95514 -779.3011 -836.6777 -...,[15402.127 15117.399 16324.493 17517.887 17517...,[76.5 6.375 6.875 29.5 29.5 31.5 ],[31.479162 31.479162 33.979145 36.47913 36.47...,[33.613575 33.613575 33.622578 33.63158 33.63...,[4. 6. 3. 9. 7. 6.],[ 4. 2. 1. 0. -2. -1.],[1. 1. 2. 3. 3. 4.],0.146734,1
4,4,23.0,4.0,9.0,10.0,1.695652,1.00,1.000000,1.200000,8,...,[-158.9613 -171.60751 -184.28438 -197.27634],[15203.427 16410.682 17604.236 18813.09 ],[ 6.375 6.875 29.5 31.5 ],[31.479162 33.979145 36.47913 38.979115],[33.613575 33.622578 33.63158 33.640583],[4. 5. 4. 3.],[-2. 1. 1. 1.],[1. 2. 3. 4.],0.147967,1


In [15]:
train.columns

Index(['Index', 'ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]',
       'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]',

In [14]:
train.shape

(12506207, 75)

In [16]:
N_TRAIN = int(1e5)

In [17]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [18]:
model.fit(train.loc[:N_TRAIN, FEATURE_COLUMNS].values,
          train.label.loc[:N_TRAIN],
          sample_weight=train.weight.loc[:N_TRAIN])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [25]:
utils.rejection10_sklearn(model, test.loc[:, FEATURE_COLUMNS].values, test.label, test.weight)

/mnt/nkazeev/miniconda3/envs/pip-gpu/lib/python3.6/site-packages/pandas/core/series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


0.7003573329204303

In [20]:
model_no_weights = xgboost.XGBClassifier(n_jobs=-1).fit(
    train.loc[:N_TRAIN, FEATURE_COLUMNS].values, train.label.loc[:N_TRAIN])

In [26]:
utils.rejection10_sklearn(model_no_weights, test.loc[:, FEATURE_COLUMNS].values, test.label, test.weight)

/mnt/nkazeev/miniconda3/envs/pip-gpu/lib/python3.6/site-packages/pandas/core/series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


0.6995551945963246